In [113]:
import pandas as pd
import numpy as np
import joblib
pd.options.mode.chained_assignment = None

test_a_base = pd.read_csv('data/test_a_base.csv')
test_a_op = pd.read_csv('data/test_a_op.csv')
test_a_trans = pd.read_csv('data/test_a_trans.csv')

In [114]:
#显示所有列
pd.set_option('display.max_columns', None)
#显示所有行
pd.set_option('display.max_rows', 100)
#设置value的显示长度为100，默认为50
pd.set_option('max_colwidth',100)

In [115]:
test_a_base_non_nan = test_a_base.drop(columns=['service3_level'],axis=1)

test_a_base_non_nan['sex'].fillna('category 0',inplace=True)
test_a_base_non_nan['provider'].fillna('category 0',inplace=True)
test_a_base_non_nan['province'].fillna('21c43413032d5522',inplace=True)
test_a_base_non_nan['city'].fillna('e3685345c54a1cef',inplace=True)

user = test_a_base_non_nan['user'].values

In [ ]:
test_base = test_a_base_non_nan
df_base = pd.DataFrame()

In [ ]:
# 年龄根据风险比例分箱
age_ratio = test_base.groupby('age')['label'].sum()/test_base.groupby('age')['label'].count()

age_1 = age_ratio[age_ratio>0.4].index.values

tmp_age = age_ratio[age_ratio<0.4]
age_2 = tmp_age[tmp_age>0.3].index.values

tmp_age = age_ratio[age_ratio<0.3]
age_3 = tmp_age[tmp_age>0.2].index.values

tmp_age = age_ratio[age_ratio<0.2]
age_4 = tmp_age[tmp_age>0.1].index.values

age_5 = age_ratio[age_ratio<0.1].index.values

def age_class(age):
    if age in age_1:
        return '0'
    elif age in age_2:
        return '1'
    elif age in age_3:
        return '2'
    elif age in age_4:
        return '3'
    elif age in age_5:
        return '4'
    else:
        return '5'
    
train_base['age_class']=train_base['age'].apply(age_class)

In [117]:
def magic_feature(df, f1, f2):
    df[f'{f1}_{f2}_a'] = df[f1]+df[f2]
    df[f'{f1}_{f2}_s'] = df[f1]-df[f2]
    df[f'{f1}_{f2}_m'] = df[f1]*df[f2]
    df[f'{f1}_{f2}_d'] = df[f1]/df[f2]


magic_feature(test_base_categoty,'city','province')
magic_feature(test_base_categoty,'balance1','balance2')
magic_feature(test_base_categoty,'balance1_avg','balance2_avg')

test_base_categoty['product_amount_a'] = test_base_categoty['product1_amount']+test_base_categoty['product2_amount']+ \
                                        test_base_categoty['product6_amount']
test_base_categoty['product_amount_m'] = test_base_categoty['product1_amount']*test_base_categoty['product2_amount']* \
                                        test_base_categoty['product6_amount']

In [118]:
df_value = test_a_base_non_user.select_dtypes('int64')

df_value['product7_success_cnt'] = df_value['product7_cnt']-df_value['product7_fail_cnt']

df_value['card_cnt'] = df_value['card_a_cnt']+df_value['card_b_cnt']+df_value['card_c_cnt']+df_value['card_d_cnt']

df_value['ip_cnt_avg'] = df_value['ip_cnt']/df_value['login_days_cnt']

df_value['login_cnt_period1_avg']=df_value['login_cnt_period1']/df_value['login_days_cnt']
df_value['login_cnt_period2_avg']=df_value['login_cnt_period2']/df_value['login_days_cnt']
df_value['login_cnt_period']=df_value['login_cnt_period1']+df_value['login_cnt_period2']
df_value['login_cnt_period_avg']=df_value['login_cnt_period']/df_value['login_days_cnt']

df_value['service_cnt']=df_value['service1_cnt']+df_value['service2_cnt']
df_value['service_avg1_amt']=df_value['service1_amt']/df_value['service1_cnt']

df_value['op_cnt']=df_value['op1_cnt']+df_value['op2_cnt']

# 归一化
df_value = (df_value-df_value.min())/(df_value.max()-df_value.min())

df_base = pd.DataFrame()
df_base['user'] = user
df_base = pd.concat([df_base, one_hot_df, test_base_categoty, df_value],axis=1)

In [119]:
df_base.shape

(24315, 86)

In [120]:
df_base.head()

,user,sex_category 0,sex_category 1,provider_category 0,provider_category 1,provider_category 2,level_category 0,level_category 1,level_category 2,verified_category 0,verified_category 1,agreement1_category 0,agreement1_category 1,agreement2_category 0,agreement2_category 1,agreement3_category 0,agreement3_category 1,agreement4_category 0,agreement4_category 1,service3_category 0,service3_category 1,product3_amount_level 1,product3_amount_level 2,product3_amount_level 3,product4_amount_level 0,product4_amount_level 1,product5_amount_level 0,product5_amount_level 1,agreement_all_0,agreement_all_1,city,province,regist_type,balance,balance_avg,balance1,balance1_avg,balance2,balance2_avg,product1_amount,product2_amount,product6_amount,city_province_a,city_province_s,city_province_m,city_province_d,balance1_balance2_a,balance1_balance2_s,balance1_balance2_m,balance1_balance2_d,balance1_avg_balance2_avg_a,balance1_avg_balance2_avg_s,balance1_avg_balance2_avg_m,balance1_avg_balance2_avg_d,product_amount_a,product_amount_m,age,using_time,card_a_cnt,card_b_cnt,card_c_cnt,op1_cnt,op2_cnt,card_d_cnt,agreement_total,service1_cnt,service1_amt,service2_cnt,acc_count,login_cnt_period1,login_cnt_period2,ip_cnt,login_cnt_avg,login_days_cnt,product7_cnt,product7_fail_cnt,product7_success_cnt,card_cnt,ip_cnt_avg,login_cnt_period1_avg,login_cnt_period2_avg,login_cnt_period,login_cnt_period_avg,service_cnt,service_avg1_amt,op_cnt
0,TestA_21427,1,0,1,0,0,0,1,0,1,0,1,0,0,1,1,0,1,0,1,0,1,0,0,1,0,1,0,0,0,0.242758,0.229435,0.272327,0.230045,0.199998,0.281048,0.296221,0.217602,0.224896,0.289505,0.289506,0.280196,0.472193,0.013324,0.055697,1.058072,0.498649,0.063446,0.061156,1.291569,0.521117,0.071325,0.066619,1.317148,0.859207,0.023484,0.344262,0.132075,0.101562,0.088235,0.065217,0.073446,0.101562,0.000000,0.422414,0.0,0.0,0.0,0.021834,0.014139,0.010796,0.049462,0.026971,0.178832,0.032787,0.0,0.857143,0.102459,0.241382,0.013135,0.007052,0.014369,0.012579,0.0,0.0,0.146893
1,TestA_08694,0,1,1,0,0,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,0,1,0,1,0,1,0,0.336491,0.361379,0.180227,0.289287,0.348267,0.281048,0.296221,0.272240,0.332418,0.289505,0.289506,0.280196,0.697870,-0.024889,0.121601,0.931128,0.553288,0.008807,0.076513,1.032351,0.628639,-0.036197,0.098469,0.891111,0.859207,0.023484,0.714286,0.698113,0.046875,0.088235,0.000000,0.033898,0.000000,0.000000,0.112069,0.0,0.0,0.0,0.016594,0.001073,0.003243,0.006989,0.076763,0.023723,0.000000,0.0,0.000000,0.049180,0.244883,0.000922,0.002831,0.001741,0.001504,0.0,0.0,0.033898
2,TestA_13449,0,1,1,0,0,0,0,1,1,0,1,0,0,1,0,1,1,0,1,0,1,0,0,1,0,1,0,0,0,0.236733,0.210135,0.180227,0.190622,0.184338,0.152312,0.193318,0.212156,0.224896,0.200390,0.151730,0.085788,0.446868,0.026599,0.049746,1.126579,0.364468,-0.059844,0.032314,0.717925,0.418214,-0.031577,0.043476,0.859591,0.437907,0.002608,0.400468,0.716981,0.101562,0.279412,0.065217,0.209040,0.000000,0.315789,0.637931,0.0,0.0,0.0,0.048035,0.102073,0.113136,0.141398,0.126556,0.479927,0.032787,0.0,0.857143,0.180328,0.244883,0.100345,0.105847,0.113175,0.109899,0.0,0.0,0.209040
3,TestA_10937,1,0,1,0,0,0,0,1,1,0,1,0,0,1,1,0,1,0,1,0,1,0,0,1,0,1,0,0,0,0.175659,0.224435,0.272327,0.239569,0.227555,0.281048,0.296221,0.238492,0.224896,0.289505,0.289506,0.280196,0.400094,-0.048776,0.039424,0.782671,0.519540,0.042556,0.067028,1.178436,0.521117,0.071325,0.066619,1.317148,0.859207,0.023484,0.512881,0.169811,0.046875,0.088235,0.000000,0.175141,0.046875,0.000000,0.267241,0.0,0.0,0.0,0.016594,0.041128,0.087069,0.134409,0.114108,0.399635,0.032787,0.0,0.857143,0.049180,0.262889,0.038936,0.080897,0.056736,0.053222,0.0,0.0,0.209040
4,TestA_14194,0,1,0,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,0,1,0,1,0,1,0,0.166661,0.229435,0.251503,0.289287,0.348267,0.281048,0.296221,0.272240,0.332418,0.289505,0.289506,0.280196,0.396095,-0.062774,0.038238,0.726396,0.553288,0.008807,0.076513,1.032351,0.628639,-0.036197,0.098469,0.891111,0.859207,0.023484,0.400468,0.000000,0.046875,0.088235,0.000000,0.000000,0.000000,0.00000

### OP

In [131]:
op_df = pd.DataFrame()
group = test_a_op.groupby(['user']).count()
op_df['user'] = group.index

count_column=test_a_op.columns[1:-1]
for column in count_column:
    print(column)
    op_df['op_'+column+'_count'] = test_a_op.groupby('user')[column].count().values
    op_df['op_'+column+'_count'] = test_a_op.groupby('user')[column].nunique().values
#     des_df = test_a_op.groupby('user')[column].describe()
#     op_df['op_'+column+'_count'] = des_df['count'].values
#     op_df['op_'+column+'_n'] = des_df['unique'].values
#     op_df['op_'+column+'_fre'] = des_df['freq'].values

op_type
op_mode
op_device
ip
net_type
channel
ip_3
tm_diff


In [132]:
from datetime import timedelta
def timedelta2sec(delta):
    day = delta.split('days')[0].strip()
    h,m,s = delta.split('days')[1].strip().split(':')
    sec=timedelta(days=int(day),hours=int(h),minutes=int(m),seconds=float(s)).total_seconds()
    return sec

test_a_op['time_diff_sec'] = test_a_op['tm_diff'].apply(timedelta2sec)

for operate in ['max','min','mean','median','std']:
    op_df['op_time_'+operate]=test_a_op.groupby('user')['time_diff_sec'].agg(operate).values
    
op_df.fillna(0,inplace=True)

In [133]:
op_df.shape

(20458, 14)

### trans

In [137]:
trans_df = pd.DataFrame()
group = test_a_trans.groupby(['user']).count()
trans_df['user'] = group.index

count_column = list(test_a_trans.columns[1:4])+list(test_a_trans.columns[5:9]) 
for column in count_column:
    print(column)
    trans_df['trans_'+column+'_count'] = test_a_trans.groupby('user')[column].count().values
    trans_df['trans_'+column+'_count'] = test_a_trans.groupby('user')[column].nunique().values
#     des_df = test_a_trans.groupby('user')[column].describe()
#     trans_df['trans_'+column+'_count'] = des_df['count'].values
#     trans_df['trans_'+column+'_n'] = des_df['unique'].values
#     trans_df['trans_'+column+'_fre'] = des_df['freq'].values

platform
tunnel_in
tunnel_out
type1
ip
type2
ip_3


In [138]:
def timedelta2sec(delta):
    day = delta.split('days')[0].strip()
    h,m,s = delta.split('days')[1].strip().split(':')
    sec=timedelta(days=int(day),hours=int(h),minutes=int(m),seconds=float(s)).total_seconds()
    return sec

In [139]:
for operate in ['max','min','mean','median','std']:
    trans_df['trans_amount_'+operate]=test_a_trans.groupby(['user'])['amount'].agg(operate).values
    
test_a_trans['time_diff_sec'] = test_a_trans['tm_diff'].apply(timedelta2sec)

for operate in ['max','min','mean','median','std']:
    trans_df['trans_time_'+operate]=test_a_trans.groupby('user')['time_diff_sec'].agg(operate).values

trans_df.fillna(0,inplace=True)

In [140]:
trans_df.shape

(18813, 18)

## 数据合并

In [67]:
df_base.shape

(24315, 74)

In [54]:
test_df = df_base

In [141]:
test_df = pd.merge(df_base, op_df, on='user', how='left')

In [142]:
test_df = pd.merge(test_df, trans_df, on='user', how='left')

In [153]:
model = joblib.load('model.pkl')

In [144]:
test_df.fillna(0,inplace=True)

In [145]:
test_df.head()

,user,sex_category 0,sex_category 1,provider_category 0,provider_category 1,provider_category 2,level_category 0,level_category 1,level_category 2,verified_category 0,verified_category 1,agreement1_category 0,agreement1_category 1,agreement2_category 0,agreement2_category 1,agreement3_category 0,agreement3_category 1,agreement4_category 0,agreement4_category 1,service3_category 0,service3_category 1,product3_amount_level 1,product3_amount_level 2,product3_amount_level 3,product4_amount_level 0,product4_amount_level 1,product5_amount_level 0,product5_amount_level 1,agreement_all_0,agreement_all_1,city,province,regist_type,balance,balance_avg,balance1,balance1_avg,balance2,balance2_avg,product1_amount,product2_amount,product6_amount,city_province_a,city_province_s,city_province_m,city_province_d,balance1_balance2_a,balance1_balance2_s,balance1_balance2_m,balance1_balance2_d,balance1_avg_balance2_avg_a,balance1_avg_balance2_avg_s,balance1_avg_balance2_avg_m,balance1_avg_balance2_avg_d,product_amount_a,product_amount_m,age,using_time,card_a_cnt,card_b_cnt,card_c_cnt,op1_cnt,op2_cnt,card_d_cnt,agreement_total,service1_cnt,service1_amt,service2_cnt,acc_count,login_cnt_period1,login_cnt_period2,ip_cnt,login_cnt_avg,login_days_cnt,product7_cnt,product7_fail_cnt,product7_success_cnt,card_cnt,ip_cnt_avg,login_cnt_period1_avg,login_cnt_period2_avg,login_cnt_period,login_cnt_period_avg,service_cnt,service_avg1_amt,op_cnt,op_op_type_count,op_op_mode_count,op_op_device_count,op_ip_count,op_net_type_count,op_channel_count,op_ip_3_count,op_tm_diff_count,op_time_max,op_time_min,op_time_mean,op_time_median,op_time_std,trans_platform_count,trans_tunnel_in_count,trans_tunnel_out_count,trans_type1_count,trans_ip_count,trans_type2_count,trans_ip_3_count,trans_amount_max,trans_amount_min,trans_amount_mean,trans_amount_median,trans_amount_std,trans_time_max,trans_time_min,trans_time_mean,trans_time_median,trans_time_std
0,TestA_21427,1,0,1,0,0,0,1,0,1,0,1,0,0,1,1,0,1,0,1,0,1,0,0,1,0,1,0,0,0,0.242758,0.229435,0.272327,0.230045,0.199998,0.281048,0.296221,0.217602,0.224896,0.289505,0.289506,0.280196,0.472193,0.013324,0.055697,1.058072,0.498649,0.063446,0.061156,1.291569,0.521117,0.071325,0.066619,1.317148,0.859207,0.023484,0.344262,0.132075,0.101562,0.088235,0.065217,0.073446,0.101562,0.000000,0.422414,0.0,0.0,0.0,0.021834,0.014139,0.010796,0.049462,0.026971,0.178832,0.032787,0.0,0.857143,0.102459,0.241382,0.013135,0.007052,0.014369,0.012579,0.0,0.0,0.146893,4.0,4.0,1.0,3.0,1.0,2.0,3.0,4.0,1016253.0,60631.0,6.681783e+05,811629.0,428653.343106,1.0,0.0,1.0,1.0,0.0,0.0,0.0,48247.0,48247.0,48247.00,48247.0,0.00000,1570268.0,1570268.0,1.570268e+06,1570268.0,0.000000
1,TestA_08694,0,1,1,0,0,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,0,1,0,1,0,1,0,0.336491,0.361379,0.180227,0.289287,0.348267,0.281048,0.296221,0.272240,0.332418,0.289505,0.289506,0.280196,0.697870,-0.024889,0.121601,0.931128,0.553288,0.008807,0.076513,1.032351,0.628639,-0.036197,0.098469,0.891111,0.859207,0.023484,0.714286,0.698113,0.046875,0.088235,0.000000,0.033898,0.000000,0.000000,0.112069,0.0,0.0,0.0,0.016594,0.001073,0.003243,0.006989,0.076763,0.023723,0.000000,0.0,0.000000,0.049180,0.244883,0.000922,0.002831,0.001741,0.001504,0.0,0.0,0.033898,13.0,16.0,1.0,2.0,3.0,3.0,2.0,36.0,1020654.0,939672.0,9.770626e+05,940384.0,40414.406946,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00000,0.0,0.0,0.000000e+00,0.0,0.000000
2,TestA_13449,0,1,1,0,0,0,0,1,1,0,1,0,0,1,0,1,1,0,1,0,1,0,0,1,0,1,0,0,0,0.236733,0.210135,0.180227,0.190622,0.184338,0.152312,0.193318,0.212156,0.224896,0.200390,0.151730,0.085788,0.446868,0.026599,0.049746,1.126579,0.364468,-0.059844,0.032314,0.717925,0.418214,-0.031577,0.043476,0.859591,0.437907,0.002608,0.400468,0.716981,0.101562,0.279412,0.065217,0.209040,0.000000,0.315789,0.637931,0.0,0.0,0.0,0.048035,0.102073,0.113136,0.141398,0.126556,0.479927,0.032787,0.0,0.857143,0.180328,0.244883,0.100345,0.105847,0.113175,0.109899,0.0,0.0,0.209040,8.0,9.0,1.0,8.0,3.0,3.0,8.0,59.0,1236753.0,14

In [146]:
test_df.shape

(24315, 116)

In [154]:
x = test_df.drop('user',axis=1)
res = model.predict_proba(x)[:,1]

In [155]:
df_res = pd.DataFrame()
df_res['user'] = test_a_base['user']
df_res['prob'] = res

In [156]:
df_res.to_csv('result.csv', index=False)

In [111]:
len(test_a_base)

24315

In [157]:
np.mean(res)

0.26766244162583336

In [60]:
np.mean(res)

0.24554811379606176

In [63]:
d = pd.read_csv('res.csv')

In [65]:
d['prob'].mean()

0.2920157286076434

In [69]:
df_res

,user,prob
0,TestA_21427,0.153202
1,TestA_08694,0.363885
2,TestA_13449,0.016144
3,TestA_10937,0.186673
4,TestA_14194,0.190885
...,...,...
24310,TestA_19183,0.211259
24311,TestA_12697,0.095210
24312,TestA_07983,0.209639
24313,TestA_01730,0.046454


In [68]:
d

,user,prob
0,TestA_21427,0.183064
1,TestA_08694,0.443422
2,TestA_13449,0.028758
3,TestA_10937,0.335119
4,TestA_14194,0.211605
...,...,...
24310,TestA_19183,0.242522
24311,TestA_12697,0.052768
24312,TestA_07983,0.272126
24313,TestA_01730,0.107049
